# African Credit Scoring Challenge - Baseline Notebook 


## <a id="1"></a>Introduction
This notebook provides a baseline approach for the African Credit Scoring Challenge. We will:
- Load and understand the credit scoring data
- Perform exploratory data analysis
- Create and evaluate a machine learning model
- Generate predictions for submission

Key aspects of this challenge:
- Predicting loan defaults in African markets
- Dealing with imbalanced data
- Building models that can generalize to new regions

In [ ]:
# === Import Libraries ===
# Data manipulation and analysis

# Settings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from scipy.stats import mode

# Advanced ML Models
from imblearn.over_sampling import BorderlineSMOTE
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier



## <a id="2"></a>Data Loading and Inspection
Let's load our training and test datasets and examine their basic properties.

In [ ]:
# Loading the train dataset
train = pd.read_csv('/data/Train.csv')
test = pd.read_csv('/data/Test.csv')
# Display the first few rows of the datasets and their shape
display("Train", train.head(), train.shape, "Test", test.head(), test.shape)

In [ ]:
# Examine data types and check for missing values
train.info()

In [ ]:
# Check for missing values in the train dataset
print(f"There are {train.isna().sum().sum()} missing values in the data.")

## <a id="4"></a>Exploratory Data Analysis (EDA)
We'll create visualization helper functions to understand our data distribution.

In [ ]:
def plot_continuous(df, column, transform=None):
    """
    Creates visualization for continuous variables with optional transformations.
    
    Parameters:
    df (DataFrame): Input dataframe
    column (str): Column name to visualize
    transform (str): Type of transformation ('log' or 'sqrt')
    """
    fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    
    # Original data plots
    sns.histplot(df[column], kde=True, ax=axs[0, 0])
    axs[0, 0].set_title(f'Distribution of {column}')
    axs[0, 0].set_xlabel(column)

    sns.boxplot(y=df[column], ax=axs[1, 0])
    axs[1, 0].set_title(f'Box Plot of {column}')
    axs[1, 0].set_ylabel(column)

    if transform == 'log':
        transformed_data = np.log1p(df[column])
        transform_label = 'Log'
    elif transform == 'sqrt':
        transformed_data = np.sqrt(df[column])
        transform_label = 'Square Root'
    else:
        transformed_data = None

    if transformed_data is not None:
        sns.histplot(transformed_data, kde=True, ax=axs[0, 1])
        axs[0, 1].set_title(f'{transform_label} Distribution of {column}')
        axs[0, 1].set_xlabel(f'{transform_label}({column})')

        sns.boxplot(y=transformed_data, ax=axs[1, 1])
        axs[1, 1].set_title(f'{transform_label} Box Plot of {column}')
        axs[1, 1].set_ylabel(f'{transform_label}({column})')
    else:
        for ax in [axs[0, 1], axs[1, 1]]:
            ax.remove()

    plt.tight_layout()
    plt.show()

def plot_categorical(df, column):
    """
    Creates visualization for categorical variables.
    
    Parameters:
    df (DataFrame): Input dataframe
    column (str): Column name to visualize
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    value_counts = df[column].value_counts()
    sns.barplot(x=value_counts.index, y=value_counts.values, ax=ax1)
    ax1.set_title(f'Bar Plot of {column}')
    ax1.set_xlabel(column)
    ax1.set_ylabel('Count')
    ax1.tick_params(axis='x', rotation=45)

    ax2.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%')
    ax2.set_title(f'Pie Chart of {column}')

    plt.tight_layout()
    plt.show()

### Categorical Variables Analysis
Let's explore the distribution of categorical variables in our dataset.

In [ ]:
# Analyze country distribution in train set
plot_categorical(train, 'country_id')

In [ ]:
# Analyze country distribution in test set
plot_categorical(test, 'country_id')

**Important Note:** The test set includes Ghana, which is not present in the training data. This indicates we need to build a model that can generalize well to new regions.

In [ ]:
# Analyze lender distribution
plot_categorical(train, 'lender_id')

### Target Variable Analysis
Let's examine our target variable distribution.

In [ ]:
# Analyze target variable distribution
plot_categorical(train, 'target')

**Key Insight:** The dataset is highly imbalanced, which is typical for loan default scenarios.

### Continuous Variables Analysis
Let's examine the distribution of numerical variables.

In [ ]:
# Analyze Total Amount distribution
plot_continuous(train, 'Total_Amount')

## <a id="5"></a>Feature Engineering
We'll combine train and test data for consistent feature engineering.

In [ ]:
# Combine datasets for consistent feature engineering
data = pd.concat([train, test]).reset_index(drop=True)

# Convert date columns to datetime
data['disbursement_date'] = pd.to_datetime(data['disbursement_date'], errors='coerce')
data['due_date'] = pd.to_datetime(data['due_date'], errors='coerce')

# update total amount to repay to the the lender portion to be repaid where total amount to be repaid is 0
data.loc[data['Total_Amount_to_Repay'] == 0, ['Total_Amount_to_Repay']] += data['Lender_portion_to_be_repaid']

# Compute mena and median values by customer for the total amount to repay
aggregates = data.groupby('customer_id')['Total_Amount_to_Repay'].agg(['mean', 'median']).reset_index()
aggregates.rename(columns={'mean': 'Mean_Total_Amount', 'median': 'Median_Total_Amount'}, inplace=True)
data=data.merge(aggregates, on='customer_id', how='left')


# Extract temporal features from dates
date_cols = ['disbursement_date', 'due_date']
for col in date_cols:
    data[col] = pd.to_datetime(data[col])
    # Extract month, day, year
    data[col+'_month'] = data[col].dt.month
    data[col+'_day'] = data[col].dt.day
    data[col+'_year'] = data[col].dt.year
    # Calculate loan term and weekday features
    data[f'loan_term_days'] = (data['due_date'] - data['disbursement_date']).dt.days
    data[f'disbursement_weekday'] = data['disbursement_date'].dt.weekday
    data[f'due_weekday'] = data['due_date'].dt.weekday

# Create some financial ratios and transformations
data['repayment_ratio'] = data['Total_Amount_to_Repay'] / data['Total_Amount']
data['amount_due_per_day'] = (data['Total_Amount_to_Repay'] / data['duration'])
data['log_Total_Amount'] = np.log1p(data['Total_Amount'])
data['log_Total_Amount_to_Repay'] = np.log1p(data['Total_Amount_to_Repay']) 
data['log_Amount_Funded_By_Lender'] = np.log1p(data['Amount_Funded_By_Lender'])
data['log_Lender_portion_to_be_repaid'] = np.log1p(data['Lender_portion_to_be_repaid'])
data['amount_to_repay_greater_than_average']=data['Mean_Total_Amount'] - data['Total_Amount_to_Repay'] 

#some outliers were noticed in the total amount and total amount to repay fields. 
# offset this by using the 90th percentile
q=0.9
data['Total_Amount_to_Repay'] = np.where(data['Total_Amount_to_Repay'] >= data['Total_Amount_to_Repay'].quantile(q), data['Total_Amount_to_Repay'].quantile(q),data['Total_Amount_to_Repay'])
data['Total_Amount'] = np.where(data['Total_Amount'] >= data['Total_Amount'].quantile(q), data['Total_Amount'].quantile(q),data['Total_Amount'])

# Handle categorical variables
cat_cols = data.select_dtypes(include='object').columns

# Label encoding for other categorical columns
le = LabelEncoder()
for col in [col for col in cat_cols if col not in ['loan_type', 'ID']]:
    data[col] = le.fit_transform(data[col])

# Split back into train and test
train_df = (data[data['ID'].isin(train['ID'].unique())]).fillna(0)
test_df = (data[data['ID'].isin(test['ID'].unique())]).fillna(0)

# Define features for modeling
features_for_modelling = [col for col in train_df.columns if col not in date_cols + ['ID', 'target', 'country_id','loan_type']]

print(f"The shape of train_df is: {train_df.shape}")
print(f"The shape of test_df is: {test_df.shape}")
print(f"The shape of train is: {train.shape}")
print(f"The shape of test is: {test.shape}")
print(f"The features for modelling are:\n{features_for_modelling}")


## <a id="6"></a>Model Development
### Cross Validation
We'll use stratified split due to class imbalance.

In [ ]:

cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

### Model Training and Optimization

In [ ]:
xgb_best_params= {'n_estimators': 250, 'max_depth': 7, 'learning_rate': 0.09952819042992249, 'subsample': 0.9138294343870095, 'colsample_bytree': 0.6808646076666579, 'gamma': 0.01070807358962328, 'min_child_weight': 1}
lgb_best_params=  {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.07587945476302646, 'num_leaves': 70, 'feature_fraction': 0.6624074561769746, 'bagging_fraction': 0.662397808134481, 'lambda_l1': 0.05808361216819946, 'lambda_l2': 0.8661761457749352}
cat_best_params= {'iterations': 500, 'depth': 7, 'learning_rate': 0.09702561867586006}


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')


# Coss validate model performance

seed=42

X,y=train_df[features_for_modelling], train_df['target']

cv_reports = []
f1_scores = []
predictions=[]
pred_prob=[]
base_estimator = DecisionTreeClassifier()
for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y),1):
    
    X_fold_train = X.iloc[train_idx]
    X_fold_valid = X.iloc[valid_idx]
    y_fold_train = y.iloc[train_idx]
    y_fold_valid = y.iloc[valid_idx]
    
    smote =BorderlineSMOTE(sampling_strategy=0.45, random_state=seed)
    
    # Apply SMOTE only on the training data of this fold
    X_fold_train_smote, y_fold_train_smote = smote.fit_resample(X_fold_train, y_fold_train)
    
    # Calculate scale_pos_weight to mitigate class imbalance
    scale_pos_weight = len(y_fold_train_smote[y_fold_train_smote == 1])*0.75 / len(y_fold_train_smote[y_fold_train_smote == 0]) 
    
    model1 = xgb.XGBClassifier(**xgb_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    model2 = LGBMClassifier(**lgb_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    model3 = CatBoostClassifier(**cat_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    

     
    # Train the model on the SMOTE-balanced fold
    model1.fit(X_fold_train_smote, y_fold_train_smote)
    model2.fit(X_fold_train_smote, y_fold_train_smote)
    model3.fit(X_fold_train_smote, y_fold_train_smote)
   

    # predict on validation set
    pred_1 = model1.predict(X_fold_valid)
    pred_2 = model2.predict(X_fold_valid)
    pred_3 = model3.predict(X_fold_valid)

    predictions = mode([pred_1,pred_2,pred_3], axis=0).mode.flatten()

    report = classification_report(y_fold_valid,predictions, output_dict=True)
    cv_reports.append(report)
        
        # Calculate and store the f1 score for this fold
    f1 = f1_score(predictions, y_fold_valid)
    f1_scores.append(f1)


    print(f"Fold {fold} Classification Report:")
    print(classification_report(y_fold_valid, predictions))
    print(f"Fold {fold} F1 Score: {f1:.4f}")
    print("-" * 50)

    # Summary of CV results
    print(f"Mean F1 Score across folds: {np.mean(f1_scores):.4f}")
  

### Test data Prediction using Models

In [ ]:

X,y=train_df[features_for_modelling], train_df['target']

predictions=[]
pred_prob=[]
base_estimator = DecisionTreeClassifier()
for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y),1):
    
    X_fold_train = X.iloc[train_idx]
    X_fold_valid = X.iloc[valid_idx]
    y_fold_train = y.iloc[train_idx]
    y_fold_valid = y.iloc[valid_idx]
    
    smote =BorderlineSMOTE(sampling_strategy=0.45, random_state=seed)
    
    # Apply SMOTE only on the training data of this fold
    X_fold_train_smote, y_fold_train_smote = smote.fit_resample(X_fold_train, y_fold_train)
        # Calculate scale_pos_weight
    scale_pos_weight = len(y_fold_train_smote[y_fold_train_smote == 1])*0.75 / len(y_fold_train_smote[y_fold_train_smote == 0]) 
    
    model1 = xgb.XGBClassifier(**xgb_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    model2 = LGBMClassifier(**lgb_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    model3 = CatBoostClassifier(**cat_best_params,scale_pos_weight=scale_pos_weight,random_state=seed)
    
     
    # Train the model on the SMOTE-balanced fold
    model1.fit(X_fold_train_smote, y_fold_train_smote)
    model2.fit(X_fold_train_smote, y_fold_train_smote)
    model3.fit(X_fold_train_smote, y_fold_train_smote)
    

    # make predictions using models
    pred_1 = model1.predict(test_df[features_for_modelling])
    pred_2 = model2.predict(test_df[features_for_modelling])
    pred_3 = model3.predict(test_df[features_for_modelling])
    pred_p_1 = model1.predict_proba(test_df[features_for_modelling])
    pred_p_2 = model2.predict_proba(test_df[features_for_modelling])
    pred_p_3 = model3.predict_proba(test_df[features_for_modelling])


    #Append predictions
    predictions.append(pred_1)
    predictions.append(pred_2)
    predictions.append(pred_3)
    pred_prob.append(pred_p_1)
    pred_prob.append(pred_p_2)
    pred_prob.append(pred_p_3)
   

In [ ]:

final_predictions = mode(predictions, axis=0).mode.flatten()
final_pred_proba=np.mean(pred_prob,axis=0)
test_df['target'] = final_predictions
test_df[['proba','proba2']]= final_pred_proba

In [ ]:
test_df.target.sum()

### Post Process the predictions for better score

In [ ]:
# concat and sort train and test data for easy post processing
train_test=pd.concat([train_df,test_df])
train_test=(train_test.sort_values(['customer_id','tbl_loan_id','disbursement_date','duration'])).reset_index(drop=True)

In [ ]:
# Post Processing functions

def check_loanid_in_train_data(tbl_loan_id):
    '''
    The function checks loans in the test data and update the target to the
    value available in the training data. The intuition behind this is that 
    a loan can only have one decision (0 or 1) and the target in the training data 
    supercedes that of the test data
    '''
    # Check if tbl_loan_id exists in the train DataFrame
    if tbl_loan_id in train_df['tbl_loan_id'].values:
        df = train_df[train_df['tbl_loan_id'] == tbl_loan_id]
        target=df.target.values[0]
    else:
        target=3  # Returns a 3 if there is no match
    return target

    
def type_3_correction (df):
    '''
    The model performed poorly in the type 3 loan type in the Ghana dataset.
    The distribution for that loan type differs to the distribution in the training data
    e.g. The repayment ratio for this loan type is generally much lower than the training dataset
    making the model to predict 0 for all instances (over 3000 instances).
    From the EDAs, it is impossible that all loans in such a class to have just one target group.
    To offset this, an assumption that loans were being disbursed to the customer untill he fails to pay was made.
    This function executes this assumption
    '''
    df1=df[df['loan_type']=='Type_3']
    customer_id=df1.customer_id.unique()
    for customer in customer_id:
        loan_id=df[df['customer_id']==customer].tail(1).tbl_loan_id.values[0]
        df.loc[df['tbl_loan_id'] == loan_id, 'target'] = 1   
    return df


def correct_loan_ids_with_conflicting_target(df):
    '''
    This function ensures that loans only have one target group (0 or 1 not both)
    even if there is more than one lender.
    '''
    # Find loan_ids with different target values
    loan_ids_with_diff_targets = df.groupby('tbl_loan_id')['target'].transform('nunique') > 1

    # Update target values to 1 for those loan_ids
    df.loc[loan_ids_with_diff_targets, 'target'] = 0
    return df    

In [ ]:
# Run through the DataFrame and apply the logic explained in the preceeding cell
for i in range(len(train_test)):
    if pd.notna(train_test.loc[i, 'proba']):
        customer_id=train_test.loc[i, 'customer_id']
            
        tbl_loan_id = train_test.loc[i, 'tbl_loan_id']
        target=check_loanid_in_train_data(tbl_loan_id)
            
        if target <2:
            train_test.loc[i, 'target']=target

train_test_f=type_3_correction (train_test)
correct_loan_ids_with_conflicting_target(train_test_f)
test_df=train_test_f[train_test_f['proba'].notna()]

In [ ]:
sub =  test_df[['ID', 'target']]
sub.head()

sub.to_csv('final_submission.csv', index=False)
sub.target.sum()

In [ ]:
import joblib

# Save models
joblib.dump(model1, '/kaggle/working/model1.pkl')
joblib.dump(model2, '/kaggle/working/model2.pkl')
joblib.dump(model3, '/kaggle/working/model3.pkl')


## <a id="1"></a>Credit Score Function
To build the credit score function, the following were considered:
- Probability of default: This is one of the models ouputs. The higher the probability, the higher the possibility of default.
- The loan history length: This is the number of historical loans granted to the customer. The higher the number, the higher the credit score.
- The number of defaults: This is the number of times the customer has defaulted. The higher the number, the lower the credit score.
- The credit score is capped between 300 and 850 to comply with the mostly used standards.
 

In [ ]:
# Group by customer_id
# calculate the number of loans and defaults

credit_score_calc = train_test_f.groupby('customer_id').agg(
    distinct_loans=('tbl_loan_id', 'nunique'),
    count_default=('target', 'sum')
).reset_index()


In [ ]:
credit_score_calc.head() 

In [ ]:

train_test_f=train_test_f.merge(credit_score_calc, on=['customer_id'])
train_test_f.head()

In [ ]:
# Calculate credit scores for the test datasets
test_df=train_test_f[train_test_f['proba'].notna()]

In [ ]:

def credit_score(prob_default, default_payment_count, loan_amount, loan_history_length):
    """
    Compute a credit score based on model output and other financial factors.
    
    :param prob_default: Probability of default (output of the model, between 0 and 1)
    :param default_payment_count: Number of defaults in payment in the customer's history
    :param loan_amount: Total_Amount requested
    :param loan_history_length: Length of loans in the customer's history
    :return: Credit score (scaled between 300 and 850)
    """
    
    # Define weights for each factor
    # The defined weights are only suggections and can be adjusted based on available data for better credit scoring
    
    weight_probab_default = -400  # High penalty for high probability of default 
    weight_default_payments = -20  # Each late payment reduces score
    weight_loan_history_length = 5  # Longer credit history increases score
    
        
    # Compute raw score
    raw_score = (
        850 + 
        weight_probab_default * prob_default +
        weight_default_payments * default_payment_count +
        weight_loan_history_length * loan_history_length
    )
    
    # Ensure score is within 300-850 range
    credit_score = np.clip(raw_score, 300, 850)
    
    return round(credit_score)



In [ ]:
#An example of how to use it with the models outputs

test_df['credit_score'] = test_df.apply(
    lambda row: credit_score(row['proba2'], row['count_default'], row['Total_Amount'], row['distinct_loans']),
    axis=1
)

In [ ]:
test_df.head()

In [ ]:
test_df.credit_score.unique()

## <a id="1"></a>Credit Score Ranges
- 300–579: Poor
- 580–669: Fair
- 670–739: Good
- 740–799: Very good
- 800–850: Excellent






## If the income of the customer is available, the function below can be used as it incorporates the income of the customer. Doing this may improve the credit score calculation

In [ ]:
def credit_score(prob_default, default_payment_count, loan_amount, loan_history_length,income):
    """
    Compute a credit score based on model output and other financial factors.
    
    :param prob_default: Probability of default (output of the model, between 0 and 1)
    :param default_payment_count: Number of defaults in payment in the customer's history
    :param loan_amount: Total_Amount requested
    :param income: The income of the customer
    :param loan_history_length: Length of loans in the customer's history
    :return: Credit score (scaled between 300 and 850)
    """
    
     # Define weights for each factor
    # The defined weights are only suggections and can be adjusted based on available data for better credit scoring
    
    weight_probab_default = -400  # High penalty for high probability of default 
    weight_default_payments = -20  # Each late payment reduces score
    weight_loan_history_length = 5  # Longer credit history increases score
    weight_loan_income_ratio = -100  # High loan relative to income reduces score
    
    
    # Compute loan-to-income ratio
    loan_income_ratio = loan_amount / (income + 1e-6)  # Avoid division by zero
    
    # Compute raw score
    raw_score = (
        850 + 
        weight_probab_default * prob_default +
        weight_default_payments * default_payment_count +
        weight_loan_income_ratio * loan_income_ratio +
        weight_loan_history_length * loan_history_length
    )
    
    # Ensure score is within 300-850 range
    credit_score = np.clip(raw_score, 300, 850)
    
    return round(credit_score)
